## **Oracle: Predict your original project's success**

In [32]:
import tensorflow as tf 
import pyaudio
import matplotlib.pyplot as plt
import matplotlib
import IPython
import struct
import numpy as np
import librosa
import librosa.display


### Option 1: Real Time Audio Capture & Waveform Visualizer

In [33]:
matplotlib.use('TkAgg') # using tkinter for visualization <3

CHUNK = 1024 * 4  # 4096 samples per chunk
FORMAT = pyaudio.paInt16
CHANNELS = 1  # mono sound because only one mic
RATE = 44100 #standard rate 44.1 khz

In [34]:
p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True, #initialize as true
    output=True, #initialize as true
    frames_per_buffer=CHUNK
)

In [35]:
# create the plot
fig, ax = plt.subplots()
plt.ion()  # interactive mode on
x = np.arange(0, 2 * CHUNK)  # x values should match the length of data_int
line, = ax.plot(x, np.random.rand(2 * CHUNK), color='pink')

In [7]:
# set y-axis limits to fit 16-bit audio
ax.set_ylim(-2000, 2000)
ax.set_xlim(0, 2 * CHUNK)

(0.0, 8192.0)

In [8]:
# set plot labels and title
ax.set_title("Real-Time Audio Waveform")
ax.set_xlabel("Sample Index")
ax.set_ylabel("Amplitude")

Text(17.972222222222214, 0.5, 'Amplitude')

In [9]:
try:
    while True:
        # read the audio data
        data = stream.read(CHUNK * 2)

        # unpack the data into 16-bit integers and center it around zero
        data_int = np.array(struct.unpack(str(2 * CHUNK) + 'h', data), dtype='int16')

        # update the line plot with new data
        line.set_ydata(data_int)

        # draw and flush the plot
        fig.canvas.draw()
        fig.canvas.flush_events()
        plt.pause(0.01)  # Small pause to give GUI time to update
except KeyboardInterrupt:
    print("Stream stopped by user.")
    stream.stop_stream()
    stream.close()
    p.terminate()

Stream stopped by user.


### Option 2: User-Uploaded Files

In [36]:
# use librosa for audio files, whereas pyaudio is used for real-time audio streams
test_audio = 'C:\\Users\\adith\\OneDrive\\Desktop\\vs code xoxo\\personal\\Oracle\\oracle_venv\\fresh-clone\\california_world.mp3'
y, sr = librosa.load(test_audio)

#load in the file: 

audio_data, sample_rate = librosa.load(test_audio, sr=None)

#print to check: 

print(f'Sample Rate: {sample_rate}')
print(f'Audio Data: {audio_data}')


#plot waveform for funzies: 

plt.figure(figsize=(10,4))
librosa.display.waveshow(audio_data, sr=sample_rate, color='pink')
plt.title('Audio Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()

Sample Rate: 44100
Audio Data: [ 0.000000e+00  0.000000e+00  0.000000e+00 ... -9.304419e-14 -9.690668e-14
 -8.942789e-14]


### Feature Extraction for the Uploaded Waveform

MFCCs: 
- a representation of short-term power spectrum of a sound 
- based on a linear cosine transform of a log of the power spectrum on a non-linear mel scale
- commonly used in speech & audio processing tasks 
- can recognize emotions in a speaker's voice and classify music into genres 

Chroma Features: 
- chroma features represent the 12 different pitch classes of a musical octave 
- really good at picking up slight changes in timbre & instrumentation 
- used in music retrieval tasks like chord recognition & key detection 
- a way to represent the tonal context of a musical audio signal in a condensed form 

Spectral Contrast: 
- measures the difference in amplitude between peaks and valleys in a sound spectrum 
- captures the relative distribution of energy across the frequency spectrum and is useful for distinguishing between different types of sounds such as speech and music

Tonnetz: 
- represent the harmonic & melodic components of music 
- captures relationships between pitches and is also useful for key detection and chord recognition

Mel-Scaled Spectrogram: 
- a spectrogram where the frequencies are scaled according the mel scale, which approximates the human ear's perception of sound 
- useful for visualizing the frequencies within an audio signal in a way that lines up with the way our ears would hear it 

In [44]:
import librosa

# reload the audio file so following code blocks can execute the && jupyter notebooks doesn't get confused
audio_path = 'C:\\Users\\adith\\OneDrive\\Desktop\\vs code xoxo\\personal\\Oracle\\oracle_venv\\fresh-clone\\california_world.mp3'
# y is the "test audio" that we had previously
y, sr = librosa.load(audio_path, sr = 22050)

In [45]:
#step 1: Extract features:

mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
chroma = librosa.feature.chroma_stft(y=y, sr=sr)
spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

In [46]:
#step 2: Combine features: 

mfccs_mean = np.mean(mfccs, axis=1)
chroma_mean = np.mean(chroma, axis=1)
spectral_contrast_mean = np.mean(spectral_contrast, axis=1)
tonnetz_mean = np.mean(tonnetz, axis=1)

In [47]:
#step 3: Combine into a single feature vector
feature_vector = np.hstack([mfccs_mean, chroma_mean, spectral_contrast_mean, tonnetz_mean])

In [48]:
#optional: print out the vector to see it for funzies
print(feature_vector)

[-1.19439873e+02  1.15534729e+02  9.10197639e+00  1.58807144e+01
  3.06296539e+00 -3.86641264e+00 -1.14549771e-01  8.58955669e+00
  6.71586752e+00  7.82180738e+00  1.80883145e+00  1.00113688e+01
  2.48616624e+00  2.90449053e-01  2.47736588e-01  3.50238234e-01
  2.78483152e-01  2.96141565e-01  5.02131581e-01  3.39676291e-01
  6.43634260e-01  3.14863056e-01  3.71400386e-01  5.44088900e-01
  3.48741293e-01  1.98347736e+01  1.44098433e+01  2.00197972e+01
  1.92636745e+01  1.83383434e+01  1.72727062e+01  4.70841222e+01
 -1.03588220e-01  1.89049989e-01  9.64397717e-02  1.87047864e-02
  4.52142891e-02  5.08466687e-02]


### Train a Model & Stuff

In [51]:
import deeplake
ds = deeplake.load("hub://activeloop/gtzan-genre")

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/gtzan-genre



|

hub://activeloop/gtzan-genre loaded successfully.



In [52]:
print(ds.summary())

Dataset(path='hub://activeloop/gtzan-genre', read_only=True, tensors=['audio', 'genre'])

 tensor      htype              shape             dtype  compression
 -------    -------            -------           -------  ------- 
  audio      audio     (1000, 660000:675808, 1)  float64    wav   
  genre   class_label         (1000, 1)          uint32    None   
None


1. extract features from the audio dataset (GTZAN)

In [56]:
# will use sklearn <3 
import numpy as np # need it to process the arrays 
import librosa # has the funcs we need like mel spectrogram etc 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [57]:
def extract_features(audio):
    mfccs = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

In [58]:
# define some vars to hold the data
X = []
y = []

In [59]:
from tqdm import tqdm # i need a progress bar i have no patience

for i in tqdm(range(len(ds))):
    audio, genre = ds[i]['audio'].numpy().flatten(), ds[i]['genre'].numpy()[0]
    features = extract_features(audio)
    X.append(features)
    y.append(genre)


  7%|▋         | 66/1000 [02:55<41:16,  2.65s/it]  


KeyboardInterrupt: 

In [23]:
X = np.array(X)
y = np.array(y)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# initalize a model 
model = RandomForestClassifier()


In [26]:
# train the model
model.fit(X_train, y_train)

RandomForestClassifier()

In [27]:
y_pred = model.predict(X_test)

In [28]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

Model Accuracy: 53.00%


In [29]:
# Predict the genre for a new audio file (assuming you have `feature_vector` already extracted)
feature_vector = extract_features(np_test_audio)
genre_prediction = model.predict([feature_vector])
print(f'Predicted Genre: {genre_prediction[0]}')

ParameterError: Audio data must be floating-point